In [2]:
#load pickles and combine to one list
import pickle
course_ids = []

for year in range(2019, 2024):
    with open('data/course_ids_' + str(year) + '.pkl', 'rb') as f:
        course_ids += pickle.load(f)

len(course_ids)

18488

In [52]:
import requests
import json
import re
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import tqdm

df = pd.DataFrame(columns=['url', 'year', 'course_title', 'language', 'college', 'lecturers', 'programmes', 'text'])

batch_size = 500
num_batches = len(course_ids) // batch_size + 1
batches = [course_ids[i:i + batch_size] for i in range(0, len(course_ids), batch_size)]

for i in tqdm(range(num_batches)):
    df = pd.DataFrame(columns=['url', 'year', 'course_title', 'language', 'college', 'lecturers', 'programmes', 'text'])
    for course in tqdm(batches[i]):

        r = requests.get(course)
        soup = BeautifulSoup(r.text, 'html.parser')

        #get =programmes,text
        url = course
        year = soup.find('div', {'id': 'title'}).find('span').text[:4]
        course_title = soup.find('div', {'id': 'title'}).find('h2').text

        try:
            info_container = soup.find('div', class_='course-data')
        except:
            info_container = None

        try:
            language = info_container.find('th', string=re.compile('(Language|Onderwijstaal)')).find_next_sibling('td').text.strip()
        except:
            language = None

        try:
            college = info_container.find('th', string=re.compile('(Faculty|Faculteit)')).find_next_sibling('td').text.strip()
        except:
            college = None

        #get lecturers
        lecturers = []
        lecturers_urls = []

        try:
            for lecturer in info_container.find('th', string=re.compile('(Coordinator|Vakcoördinator)')).find_next_siblings('td'):
                lecturers.append(lecturer.text.strip())
        except:
            pass
        try:
            for lecturer in info_container.find('th', string=re.compile('(Examiner|Examinator)')).find_next_siblings('td'):
                lecturers.append(lecturer.text.strip())
        except:
            pass

        try:
            for lecturer in info_container.find('th', string=re.compile('(Teaching Staff|Docenten)')).find_next_siblings('td'):
                text = lecturer.text.strip()
                #split on newline
                text = text.split('\n')

                lecturers.extend(text)
        except:
            pass
        
        lecturers = set(lecturers)
        lecturers = list(lecturers)

        programmes = []

        try:
            text = soup.find('div', {'id': 'course-description'}).text.strip()
            text = text.split('\n')
            text = [line.strip() for line in text]
            text = [line for line in text if line != '']

        except:
            text = None

        
        #make dict
        course_dict = {
            'url': url,
            'year': year,
            'course_title': course_title,
            'language': language,
            'college': college,
            'lecturers': lecturers,
            'programmes': programmes,
            'text': text
        }

        #append to df
        df = df._append(course_dict, ignore_index=True)

    #save df to csv
    df.to_csv('data/course_details_' + str(i) + '.csv', index=False)



  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

...

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/488 [00:00<?, ?it/s]